In [1]:
import pandas as pd 
import numpy as np
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from tensorflow import keras
import sys
import PIL
import os
import tensorflow.keras.backend as K
import glob
sys.path.append('../scripts/helper_functions_cv/tensorflow_helpers/')
from dotenv import load_dotenv
from sklearn.model_selection import train_test_split
import segmentation_models as sm

/home/ubuntu/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


Segmentation Models: using `keras` framework.


In [2]:
csv_file = pd.read_csv('../../ml-data-training/ship_segmentation_data/train_ship_segmentations_v2.csv')
csv_file = csv_file.groupby('ImageId')['EncodedPixels'].apply(list).reset_index()
image_ids, pixels = csv_file['ImageId'].values.tolist(), csv_file['EncodedPixels'].values.tolist()

In [3]:
csv_file['fixed_inputs'] = csv_file['ImageId'].apply(lambda x: '../../ml-data-training/ship_segmentation_data/train_v2/' + x)
csv_file['mask_paths'] = csv_file['ImageId'].apply(lambda x: '../../ml-data-training/ship_segmentation_data/masks_v1/train/' + x.split('.')[0] + '.' + 'png')

In [4]:
for x in csv_file['fixed_inputs'].values.tolist():
    if os.path.exists(x) == False:
        print(x)

In [5]:
for x in csv_file['mask_paths'].values.tolist():
    if os.path.exists(x) == False:
        print(x)

Input and masks path checks are done.

In [6]:
allowed_gpus = [0]
gpus = tf.config.list_physical_devices("GPU")
final_gpu_list = [gpus[x] for x in allowed_gpus]
tf.config.set_visible_devices(final_gpu_list, "GPU")

strategy = tf.distribute.MirroredStrategy()
AUTO = tf.data.experimental.AUTOTUNE
REPLICAS = strategy.num_replicas_in_sync

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [7]:
print(REPLICAS)

1


In [8]:
load_dotenv('../config_files/dev.env')

True

In [9]:
def split_datasets(csv_file, test_size = 0.01):
    train, test = train_test_split(csv_file, test_size = test_size)
    train, val = train_test_split(train, test_size = test_size)
    return train, val, test

In [10]:
train, val, test = split_datasets(csv_file)

In [11]:
def read_train_imgs(img, mask, shape):
    img = tf.io.read_file(img)
    mask = tf.io.read_file(mask)
    img = tf.image.decode_jpeg(img, channels=3)
    mask = tf.image.decode_jpeg(mask, channels=1)
    img = tf.image.resize(img, size=shape)
    mask = tf.image.resize(mask, size=shape)
    img = img / 255
    mask = mask / 255
    return img, mask

def get_data(data, shape = (256, 256), shuffle = True, repeat = True, batch = True, batch_size = 32):
    imgs, masks = data['fixed_inputs'].values.tolist(), data['mask_paths'].values.tolist()
    shapes = [shape for x in range(len(imgs))]
    tensor = tf.data.Dataset.from_tensor_slices((imgs, masks, shapes))
    tensor = tensor.cache()
    if repeat:
        tensor = tensor.repeat()
    if shuffle:
        tensor = tensor.shuffle(1024 * 1)
        opt = tf.data.Options()
        opt.experimental_deterministic = False
        tensor = tensor.with_options(opt)
    tensor = tensor.map(read_train_imgs)
    if batch:
        tensor = tensor.batch(batch_size * REPLICAS)
    tensor = tensor.prefetch(AUTO)
    return tensor

In [12]:
train_dataset = get_data(train, shape = (32, 32), batch_size=32)
val_dataset = get_data(val, shape = (32, 32), batch_size=32)

In [13]:
def get_model(model_name):
    with strategy.scope():
        model = sm.Unet(model_name)
        model.compile(
            'SGD',
            loss = sm.losses.bce_jaccard_loss,
            metrics = [sm.metrics.iou_score]
        )
    return model

In [15]:
model = get_model('resnet50')
model.fit(
    train_dataset,
    steps_per_epoch = len(train) // (32 * REPLICAS),
    epochs = 1,
    verbose = 1,
    validation_data = val_dataset
)

2057/5897 [=========>....................] - ETA: 2:58 - loss: 1.0232 - iou_score: 9.8682e-04

InvalidArgumentError: Graph execution error:

2 root error(s) found.
  (0) INVALID_ARGUMENT:  jpeg::Uncompress failed. Invalid JPEG data or crop window.
	 [[{{node DecodeJpeg}}]]
	 [[MultiDeviceIteratorGetNextFromShard]]
	 [[RemoteCall]]
	 [[IteratorGetNext]]
	 [[div_no_nan_1/ReadVariableOp_1/_140]]
  (1) INVALID_ARGUMENT:  jpeg::Uncompress failed. Invalid JPEG data or crop window.
	 [[{{node DecodeJpeg}}]]
	 [[MultiDeviceIteratorGetNextFromShard]]
	 [[RemoteCall]]
	 [[IteratorGetNext]]
0 successful operations.
0 derived errors ignored. [Op:__inference_train_function_33998]